In [1]:
!pip install lxml

In [2]:
# Bibliotecas

import requests           # Pegar os links para as planilhas & Baixar as planilhas propriamente ditas

pag = "https://www1.folha.uol.com.br/cotidiano/2022/01/apos-reportagem-da-folha-justica-absolve-lavrador-que-ficou-preso-por-7-anos.shtml"

#pag = "https://www1.folha.uol.com.br/mundo/2022/01/coreia-do-norte-diz-que-fez-novo-teste-de-missil-hipersonico.shtml?origin=folha"

resposta = requests.get(pag)
html = resposta.content

# Encoding
html = html.decode("UTF-8")

In [3]:
from lxml.html import document_fromstring

arvore = document_fromstring(html)

In [4]:
import re

tags_head = arvore.xpath("//title")

title_info = []
for tag in tags_head:
  tit = tag.xpath("text()")[-0]
  tit = str(tit)
  if re.findall('\d{2}/\d{2}/\d{4}', tit):
    title_info.append(tit) 

In [5]:
# Get some parameters
param = title_info[0].split("-")

title = param[0].strip()
date = param[1].strip()
edit = param[2].strip()

In [6]:
# Subtitle
## <meta name="description" content="José Aparecido Alves Filho havia sido condenado a 21 anos por latrocínio">
tags_subtit = arvore.xpath('//meta[@name="description"]')

subtitle = []
for tag in tags_subtit:
  subtit = tag.attrib.get("content")
  subtitle.append(subtit)

subtitle = subtitle[0].strip()

In [7]:
subtitle

'José Aparecido Alves Filho havia sido condenado a 21 anos por latrocínio'

In [8]:
# Author
tags_author = arvore.xpath('//div[@class="c-signature"]/strong/a')

author_aux = []
for tag in tags_author:
    assin = tag.xpath("text()")
    author_aux.append(assin)

In [9]:
author_aux

[['Rogério Pagnan'], ['Artur Rodrigues']]

In [10]:
# In case of more than one author:
def concatenate_authors(list):
    result_aux = ''
    for element in list:
        result_aux += ' AND ' + str(element[0])

    nchar = len(result_aux)
    result = result_aux[5:nchar]    
    return result


author = concatenate_authors(author_aux) 
author

'Rogério Pagnan AND Artur Rodrigues'

In [11]:
# ArticleID
def regex_id(string):
  pattern = re.compile(r'"\d{16}"') 
  result = pattern.match(string)
  if result:
    return  result.group()
  return None


html_split = html.split('"articleId": ')
articleId_aux = html_split[1]
 
articleId = regex_id(articleId_aux)[1:16]

In [12]:
# https://www.w3resource.com/python-exercises/python-basic-exercise-27.php
def concatenate_list_data(list):
    result = ''
    for element in list:
        result += str(element)
    return result

In [13]:
# https://stackoverflow.com/questions/17380869/get-list-items-inside-div-tag-using-xpath
tags_p = arvore.xpath('//div[@class="c-news__body"]//p')

#import codecs

paragrafo = []

for tag in tags_p:
  parag = tag.xpath("text()")
  if len(parag):
    parag_final = concatenate_list_data(parag)
    paragrafo.append(parag_final)

In [14]:
paragrafo

['A Justiça de São Paulo absolveu o trabalhador rural , que ficou preso sete anos. Ele foi solto em julho do ano passado, por determinação do , após a contar a história dele .',
 'A sentença absolutória do juiz Lucas Pereira Moraes Garcia, da 1ª Vara Criminal de Bragança Paulista, foi publicada na última sexta-feira (7). Como se trata de uma decisão de primeira instância, o Ministério Público ainda pode recorrer.',
 'À , José Aparecido disse que a decisão tira um peso das costas dele. "Nesse tempo, estava pensando no que iria acontecer. Estava meio inseguro. Agradeço a Deus, estou muito feliz", disse ele, que é casado e tem dois filhos.',
 'O lavrador afirma que, um mês após ser libertado, já recomeçou a vida de trabalho rural com um antigo patrão que o procurou. Ele disse ainda ter sido bem recebido na vizinhança. "Todo mundo acredita na minha inocência, no bairro. Todo mundo me abraçou, confiam em mim."',
 'Agora, ele trabalha para reconstruir tudo que perdeu nos sete anos que ficou 

In [15]:
text = concatenate_list_data(paragrafo) 
text

'A Justiça de São Paulo absolveu o trabalhador rural , que ficou preso sete anos. Ele foi solto em julho do ano passado, por determinação do , após a contar a história dele .A sentença absolutória do juiz Lucas Pereira Moraes Garcia, da 1ª Vara Criminal de Bragança Paulista, foi publicada na última sexta-feira (7). Como se trata de uma decisão de primeira instância, o Ministério Público ainda pode recorrer.À , José Aparecido disse que a decisão tira um peso das costas dele. "Nesse tempo, estava pensando no que iria acontecer. Estava meio inseguro. Agradeço a Deus, estou muito feliz", disse ele, que é casado e tem dois filhos.O lavrador afirma que, um mês após ser libertado, já recomeçou a vida de trabalho rural com um antigo patrão que o procurou. Ele disse ainda ter sido bem recebido na vizinhança. "Todo mundo acredita na minha inocência, no bairro. Todo mundo me abraçou, confiam em mim."Agora, ele trabalha para reconstruir tudo que perdeu nos sete anos que ficou preso, quando sua fam

In [16]:
articleId

'172159811189613'

In [17]:
# To save data with its important information
def save_data(TITLE, SUBTITLE, TEXT, ARTICLEID, AUTHOR, DATE, EDIT, PATH_SAVE):

  text_final = TITLE + '. ' + SUBTITLE + '. ' + TEXT

  # Write line to file
  name_save = str(ARTICLEID) + '_' + AUTHOR + '_' + DATE + '_' + EDIT  + '.txt' 
  
  with open(PATH_SAVE + name_save, 'w') as writefile:
    writefile.write(text_final)

In [18]:
# Google drive conection
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
save_data(title, 
          subtitle, 
          text, 
          articleId, 
          author, 
          date, 
          edit, 
          '/content/drive/MyDrive/INSPER/03_trabfinal_1certif/Dados/Processed/')

FileNotFoundError: ignored